# TPC QA access demo - extended version

require Root/libStat (AliRoot) to be installed togother with RootInteractive Example Notebook to demonstrate bokehDraw wrapper for the interactive visualization of treee
## Content
* 0.) Load libraries
* 1.) Read data from the http server and read the tree - or set of trees
* 2.) Dump list of variables
* 3.) TPC QA Alarm decomposition example
  * 3.a) TPC dEdx standard QA alarm
  * 3.b)  TPC global warning automatic alarm decomposition
* 4.) PID visualization example
* 5.)  Additional functionality DEMO
  *  5.a)  Example brach find  
  *  5.b)  Example parsing of the variables in formula
  *  5.c)  Example find the list of variable needed for boke graphics parsing slectionmvaraible draw widgets

### 0.) Load libraries

In [ ]:
from RootInteractive.TTreeHnInteractive.TTreeHnBrowser import *
from RootInteractive.InteractiveDrawing.bokeh.bokehDraw import *
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawPanda import *
from RootInteractive.Tools.aliTreePlayer import *
output_notebook()
import ROOT
ROOT.gSystem.Load("$ALICE_ROOT/lib/libSTAT.so")
from RootInteractive.Tools.aliTreePlayer import *
import pyparsing 
from bokeh.io import curdoc
curdoc().theme = 'caliber'

###  1.) Read data from the http server and read the tree - or set of trees
* to check interface - type name of function and ? - e.g.: ROOT.AliTreePlayer.LoadTrees?
* some manipulation with tree t get metadata content in anyTree format  - to be formalized/encapuslated
* define list of aliases in browasable format

In [ ]:
ROOT.TFile.SetCacheFileDir("../data/")
treeQA, treeList, fileList = LoadTrees("echo https://aliqat.web.cern.ch/aliqat/qcml/data/2018/LHC18q/trending_merged_LHC18q_withStatusTree.root",".*",".*sta.*",".*",0)
treeQA.RemoveFriend(treeQA.GetFriend("Tstatus"))
treeQA.RemoveFriend(treeQA.GetFriend("tpcQA"))
AddMetadata(treeQA,"chunkBegin.isTime","1")
AddMetadata(treeQA,"chunkMedian.isTime","1")
# this lines are neaded in case automatically generated parameter list  (expert mode)
branchTree=treeToAnyTree(treeQA)
aliases=aliasToDictionary(treeQA)

###  2.) Dump list of variables
There are several ways to query the content of the tree. Below you can check standart ways implemented by ROOT
To test differnt ways, please check and uncoment lises below

In [ ]:
treeQA.Show(0)
#treeQA.GetListOfAliases().Print("","*er*")
#treeQA.SetAlias("globalWarning","global_Warning")
#treeQA.GetListOfBranches().Print("","*MIP*")
#treeQA.GetListOfBranches().Print("","*gr*")

###  3.) Alarm decomposition example

#### 3.a) TPC dEdx standard QA alarm

In [ ]:
base=makeAliasAnyTree("MIPquality_Warning",aliases)
print(RenderTree(base))
#DotExporter(base).to_picture("MIPquality_Warning.png")

#### 3.b)  TPC global warning automatic alarm decomposition

In [ ]:
base=makeAliasAnyTree("global_Warning",aliases)
print(RenderTree(base))
#DotExporter(base).to_picture("Warning.png")

### 4.)  PID visualization example
* visualizing TPC QA in 5 minutes interval
* color code - status - red- warning, blue - normal
* variable to draw: 
  * MIP position, Electon position, ratio
* widgtets:
  * sliders tab - to select ranges
  * checkbox for global status
* toltips:
  * text to be shown on hower
  
TODO:
  * Add wrapper for status bars
  * On click discplay decision tree for given point

In [ ]:
bokehDraw.verbose=0
selection="meanMIP>0&resolutionMIP>0&time>0"
treeQA.SetAlias("MIPElectronRatio","meanMIP/meanMIPele")
varDraw="meanMIP:meanMIPele:MIPElectronRatio:resolutionMIP"
tooltips=[("MIP","(@meanMIP)"),  ("Electron","@meanMIPele"), ("Global status","(@global_Outlier,@global_Warning)"), \
          ("MIP status(Warning,Outlier,Acc.)","@MIPquality_Warning,@MIPquality_Outlier,@MIPquality_PhysAcc")]
for warning in getAliasAnyTree(makeAliasAnyTree("MIPquality_Warning",aliases),".*",maxlevel=2):
    tooltips.append((warning,"@"+warning))

widgets="tab.sliders(slider.meanMIP(45,55,0.1,45,55),slider.meanMIPele(50,80,0.2,50,80), slider.resolutionMIP(0,0.15,0.01,0,0.15)),"
widgets+="tab.checkboxGlobal(slider.global_Warning(0,1,1,0,1),checkbox.global_Outlier(0)),"
widgets+="tab.checkboxMIP(slider.MIPquality_Warning(0,1,1,0,1),checkbox.MIPquality_Outlier(0), checkbox.MIPquality_PhysAcc(1))"
layout="((0,1),(2),(3,x_visible=1,plot_height=200),commonX=2,x_visible=0,plot_height=350,plot_width=1000)"
xxx=bokehDraw(treeQA,selection,"chunkMedian",varDraw,"MIPquality_Warning",widgets,0,commonX=1,size=4,tooltip=tooltips,x_axis_type='datetime',layout=layout)

In [ ]:
# bokehDraw?  # Example - inable this line to find list of options

## 5.) Additional functionality DEMO

### 5.a)  Example brach find

In [ ]:
branchTree=treeToAnyTree(treeQA)
print(findSelectedBranch(branchTree,"bz"))
# custom find
for a in findall(branchTree, filter_=lambda node: re.match("gr.*Ncl.*", node.name)): print(a.name)
# predefined find with regular expresion -the same as above
for a in findSelectedBranch(branchTree,"gr.*Ncl.*"):print(a.name)

### 5.b)  Example parsing of the variables in formula

In [ ]:
parseTreeVariables("x>1 & x>0 | y==1 |x+1>2| (x2<2) | (x1*2)<2| sin(x)<1")

### 5.c  Example find the list of variable needed for boke graphics parsing slectionmvaraible draw widgets
* dumy variable added to the list. This varaible is rejected as it is not accessible in tree (leaf, alias, friend...) 



In [ ]:
toRemove=["^tab\..*"]
toReplace=["^slider.","^checkbox."]
getAndTestVariableList([selection,varDraw,widgets,"xxx"],toRemove,toReplace,treeQA)

In [ ]:
print(findSelectedBranch(branchTree,".*ert.*"))

In [ ]:
findSelectedBranch(branchTree,".*")